In [ ]:
!pip install -r requirements.txt -q

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
conversation_index = f"{dataset}-conversational"

In [ ]:
transform_config = {
    "source": {
        "index": [f"{dataset}-runtime", f"{dataset}-conversational", f"{dataset}-context", f"{dataset}-humanity"]
    },
    "dest": {
        "index": f"{dataset}"
    },
    "pivot": {
        "group_by": {
            "session_id": {
                "terms": {
                    "field": "session_id"
                }
            },
            "qa_id": {
                "terms": {
                    "field": "qa_id"
                }
            }
        },
        "aggregations": {
            "assistant": {
                "scripted_metric": {
                    "init_script": "state.assistant = []",
                    "map_script": "if (doc.containsKey('assistant') && !doc['assistant'].empty) { state.assistant.add(doc['assistant'].value) }",
                    "combine_script": "return state.assistant",
                    "reduce_script": "def list = []; for (s in states) { list.addAll(s) } return list"
                }
            }
        }
    }
}

In [ ]:
response = es.transport.perform_request(
    "PUT", 
    "/_transform/transform_conversacion_unificada", 
    body=json.dumps(transform_config)
)
print("Transform creado:", response)

In [ ]:
start_response = es.transport.perform_request(
    "POST",
    "/_transform/transform_conversacion_unificada/_start"
)
print("Transform iniciado:", start_response)